In [1]:
import os
import sys

import scipy as sp
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from pyod.models.iforest import IForest
from pyod.models.lof import LOF
from pyod.models.knn import KNN
from pyod.models.auto_encoder import AutoEncoder
from pyod.utils.data import evaluate_print

from combo.models.score_comb import majority_vote, maximization, average

# suppress warnings
import warnings

warnings.filterwarnings("ignore")

# temporary solution for relative imports in case combo is not installed
# if combo is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

from suod.models.base import SUOD
from suod.utils.utility import get_estimators_small

Using TensorFlow backend.


In [ ]:
### separate pet/ct positive/negative models

In [4]:
from imblearn.over_sampling import SMOTE 
from imblearn.over_sampling import SVMSMOTE
# load files
for repeat in range(1,6):
    for CV in range(1,6):
        mat_file_list = [
        '../../../data/Models/Cli/Cli3/Cli_Model_CV_'+str(repeat)+'_Fold_'+str(CV)+'.mat',
        '../../../data/Models/PET/PET_Model_CV_'+str(repeat)+'_Fold_'+str(CV)+'.mat',
        '../../../data/Models/CT/CT_Model_CV_'+str(repeat)+'_Fold_'+str(CV)+'.mat']
        print(mat_file_list)
        
        mat_file = mat_file_list[1]
        mat_file_name = mat_file.replace('.mat', '')
        print("\n... Processing", mat_file_name, '...')
        mat = sp.io.loadmat(mat_file)
        PET1 = mat['PET_sam_train_data0']
        PET2 = mat['PET_sam_val_data']
        PET = np.concatenate((PET1, PET2))
        print('PET_feature shape is', PET.shape)
        PET_test = mat['PET_sam_test_data']

        mat_file = mat_file_list[2]
        mat_file_name = mat_file.replace('.mat', '')
        print("\n... Processing", mat_file_name, '...')
        mat = sp.io.loadmat(mat_file)
        CT1 = mat['CT_sam_train_data0']
        CT2 = mat['CT_sam_val_data']
        CT = np.concatenate((CT1, CT2))
        print('CT_feature shape is', CT.shape)
        CT_test = mat['CT_sam_test_data']
        
        # load label
        Ind1 = mat['CT_ind_train_data0']
        Ind2 = mat['CT_ind_val_data']-1
        Ind0 = np.concatenate((Ind1, Ind2))
        Ind_test = mat['CT_ind_test_data']-1
        
        # combine feature
        Feature_train = PET
        print('\nFeature shape is', Feature_train.shape)
        print('\nLabel shape is', Ind0.shape)

        Feature_test = PET_test
        print('\nFeature test shape is', Feature_test.shape)
        print('\nLabel test shape is', Ind_test.shape)
        
        # SMOTE
        sm = SVMSMOTE(random_state=123)

        Feature, Ind = sm.fit_resample(Feature_train, Ind0)
        print('\nAfter SMOTE feature shape is', Feature.shape)
        
        
        
        # positive model
        Feature_pos = Feature[Ind==1,:]
#         Ind_pos = 
        
        # model training
        clf = AutoEncoder(hidden_neurons = [32, 16, 4, 4, 16, 32],epochs=100)
        clf.fit(Feature_pos)
        
        
        # model testing
        y_pred = clf.predict_proba(Feature_test)  # predict scores
        
        y_pred_train = clf.predict_proba(PET[Ind0[:,0]==1,:])  # predict scores
        
        
        with open('AnomalyScore/AutoEncoder_P_PET_Repeat_'+str(repeat)+'CV_'+str(CV)+'.npy', 'wb') as f:
            np.save(f,y_pred)
        
        with open('AnomalyScore/AutoEncoder_P_PET_Train_Repeat_'+str(repeat)+'CV_'+str(CV)+'.npy', 'wb') as f:
            np.save(f,y_pred_train)
            
        
        # negative model
        Feature_neg = Feature[Ind==0,:]
#         Ind_neg = 
        
        # model training
        clf = AutoEncoder(hidden_neurons = [32, 16, 4, 4, 16, 32],epochs=100)
        clf.fit(Feature_neg)
        
        
        # model testing
        y_pred = clf.predict_proba(Feature_test)  # predict scores
        
        y_pred_train = clf.predict_proba(PET[Ind0[:,0]==0,:])  # predict scores
        
        
        with open('AnomalyScore/AutoEncoder_N_PET_Repeat_'+str(repeat)+'CV_'+str(CV)+'.npy', 'wb') as f:
            np.save(f,y_pred)
        
        with open('AnomalyScore/AutoEncoder_N_PET_Train_Repeat_'+str(repeat)+'CV_'+str(CV)+'.npy', 'wb') as f:
            np.save(f,y_pred_train)
            
            
        # combine feature
        Feature_train = CT
        print('\nFeature shape is', Feature_train.shape)

        Feature_test = CT_test
        print('\nFeature test shape is', Feature_test.shape)
        
        # SMOTE
#         sm = SVMSMOTE(random_state=112)
        sm = SVMSMOTE(random_state=123)
#         sm = SVMSMOTE(random_state=111)

        Feature, Ind = sm.fit_resample(Feature_train, Ind0)
        print('\nAfter SMOTE feature shape is', Feature.shape)
        
        
        
        # positive model
        Feature_pos = Feature[Ind==1,:]
#         Ind_pos = 
        
        # model training
        clf = AutoEncoder(hidden_neurons = [32, 16, 4, 4, 16, 32],epochs=100)
        clf.fit(Feature_pos)
        
        
        # model testing
        y_pred = clf.predict_proba(Feature_test)  # predict scores
        
        y_pred_train = clf.predict_proba(CT[Ind0[:,0]==1,:])  # predict scores
        
        
        with open('AnomalyScore/AutoEncoder_P_CT_Repeat_'+str(repeat)+'CV_'+str(CV)+'.npy', 'wb') as f:
            np.save(f,y_pred)
        
        with open('AnomalyScore/AutoEncoder_P_CT_Train_Repeat_'+str(repeat)+'CV_'+str(CV)+'.npy', 'wb') as f:
            np.save(f,y_pred_train)
            
        
        # negative model
        Feature_neg = Feature[Ind==0,:]
#         Ind_neg = 
        
        # model training
        clf = AutoEncoder(hidden_neurons = [32, 16, 4, 4, 16, 32],epochs=100)
        clf.fit(Feature_neg)
        
        
        # model testing
        y_pred = clf.predict_proba(Feature_test)  # predict scores
        
        y_pred_train = clf.predict_proba(CT[Ind0[:,0]==0,:])  # predict scores
        
        
        with open('AnomalyScore/AutoEncoder_N_CT_Repeat_'+str(repeat)+'CV_'+str(CV)+'.npy', 'wb') as f:
            np.save(f,y_pred)
        
        with open('AnomalyScore/AutoEncoder_N_CT_Train_Repeat_'+str(repeat)+'CV_'+str(CV)+'.npy', 'wb') as f:
            np.save(f,y_pred_train)

['../Cli3/UTSW_Cli_224P_OrderFea_10_SMOTE_CV_it_2_10_1.mat', '../PrePost/PET_MOO/HN_MR_1_PPD_SMOTE_CV5_fea50_it_50_1.mat', '../PrePost/CT_MOO/HN_MR_1_PPD_SMOTE_CV5_fea50_val_it_50_1.mat']

... Processing ../PrePost/PET_MOO/HN_MR_1_PPD_SMOTE_CV5_fea50_it_50_1 ...
PET_feature shape is (179, 50)

... Processing ../PrePost/CT_MOO/HN_MR_1_PPD_SMOTE_CV5_fea50_val_it_50_1 ...
CT_feature shape is (179, 50)

Feature shape is (179, 50)

Label shape is (179, 1)

Feature test shape is (45, 50)

Label test shape is (45, 1)

After SMOTE feature shape is (266, 50)
Model: "sequential_201"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1401 (Dense)           (None, 50)                2550      
_________________________________________________________________
dropout_1201 (Dropout)       (None, 50)                0         
_________________________________________________________________
dense_1402 (Dense)      

119/119 [==============================] - 0s 2ms/step - loss: 14.9958 - val_loss: 7.7641
Epoch 50/100
119/119 [==============================] - 0s 2ms/step - loss: 14.6563 - val_loss: 7.6067
Epoch 51/100
119/119 [==============================] - 0s 2ms/step - loss: 14.4769 - val_loss: 7.4514
Epoch 52/100
119/119 [==============================] - 0s 2ms/step - loss: 14.1571 - val_loss: 7.2991
Epoch 53/100
119/119 [==============================] - 0s 2ms/step - loss: 13.8297 - val_loss: 7.1575
Epoch 54/100
119/119 [==============================] - 0s 2ms/step - loss: 13.5818 - val_loss: 7.0142
Epoch 55/100
119/119 [==============================] - 0s 2ms/step - loss: 13.2379 - val_loss: 6.8817
Epoch 56/100
119/119 [==============================] - 0s 2ms/step - loss: 12.9330 - val_loss: 6.7530
Epoch 57/100
119/119 [==============================] - 0s 2ms/step - loss: 12.6248 - val_loss: 6.6230
Epoch 58/100
119/119 [==============================] - 0s 2ms/step - loss: 12.4721 - 

119/119 [==============================] - 0s 2ms/step - loss: 9.8443 - val_loss: 5.1439
Epoch 83/100
119/119 [==============================] - 0s 2ms/step - loss: 9.5052 - val_loss: 5.0957
Epoch 84/100
119/119 [==============================] - 0s 2ms/step - loss: 9.5574 - val_loss: 5.0453
Epoch 85/100
119/119 [==============================] - 0s 2ms/step - loss: 9.4596 - val_loss: 4.9957
Epoch 86/100
119/119 [==============================] - 0s 2ms/step - loss: 9.4042 - val_loss: 4.9454
Epoch 87/100
119/119 [==============================] - 0s 2ms/step - loss: 9.0724 - val_loss: 4.8989
Epoch 88/100
119/119 [==============================] - 0s 2ms/step - loss: 9.2050 - val_loss: 4.8536
Epoch 89/100
119/119 [==============================] - 0s 2ms/step - loss: 9.0439 - val_loss: 4.8111
Epoch 90/100
119/119 [==============================] - 0s 2ms/step - loss: 8.7015 - val_loss: 4.7669
Epoch 91/100
119/119 [==============================] - 0s 2ms/step - loss: 8.6824 - val_loss: 

Epoch 35/100
119/119 [==============================] - 0s 2ms/step - loss: 27.8200 - val_loss: 13.6951
Epoch 36/100
119/119 [==============================] - 0s 2ms/step - loss: 27.1306 - val_loss: 13.2103
Epoch 37/100
119/119 [==============================] - 0s 2ms/step - loss: 26.3637 - val_loss: 12.7617
Epoch 38/100
119/119 [==============================] - 0s 2ms/step - loss: 25.0898 - val_loss: 12.3587
Epoch 39/100
119/119 [==============================] - 0s 2ms/step - loss: 24.5380 - val_loss: 11.9676
Epoch 40/100
119/119 [==============================] - 0s 2ms/step - loss: 23.6553 - val_loss: 11.6151
Epoch 41/100
119/119 [==============================] - 0s 2ms/step - loss: 23.2312 - val_loss: 11.2964
Epoch 42/100
119/119 [==============================] - 0s 2ms/step - loss: 22.8006 - val_loss: 11.0055
Epoch 43/100
119/119 [==============================] - 0s 2ms/step - loss: 21.7754 - val_loss: 10.7419
Epoch 44/100
119/119 [==============================] - 0s 2ms/s

Train on 119 samples, validate on 14 samples
Epoch 1/100
119/119 [==============================] - 141s 1s/step - loss: 208.6458 - val_loss: 61.9621
Epoch 2/100
119/119 [==============================] - 0s 2ms/step - loss: 171.6245 - val_loss: 54.1623
Epoch 3/100
119/119 [==============================] - 0s 2ms/step - loss: 144.8616 - val_loss: 48.6527
Epoch 4/100
119/119 [==============================] - 0s 2ms/step - loss: 127.4976 - val_loss: 44.4373
Epoch 5/100
119/119 [==============================] - 0s 2ms/step - loss: 116.4942 - val_loss: 41.0757
Epoch 6/100
119/119 [==============================] - 0s 2ms/step - loss: 105.2558 - val_loss: 38.3619
Epoch 7/100
119/119 [==============================] - 0s 2ms/step - loss: 96.9714 - val_loss: 36.1598
Epoch 8/100
119/119 [==============================] - 0s 2ms/step - loss: 88.4082 - val_loss: 34.2628
Epoch 9/100
119/119 [==============================] - 0s 2ms/step - loss: 85.0798 - val_loss: 32.5966
Epoch 10/100
119/119 

...

Train on 119 samples, validate on 14 samples
Epoch 1/100
119/119 [==============================] - 295s 2s/step - loss: 209.2883 - val_loss: 71.2495
Epoch 2/100
119/119 [==============================] - 0s 4ms/step - loss: 173.7582 - val_loss: 61.9339
Epoch 3/100
119/119 [==============================] - 0s 4ms/step - loss: 151.2504 - val_loss: 55.0710
Epoch 4/100
119/119 [==============================] - 0s 4ms/step - loss: 133.2264 - val_loss: 49.9383
Epoch 5/100
119/119 [==============================] - 0s 4ms/step - loss: 115.3051 - val_loss: 46.0170
Epoch 6/100
119/119 [==============================] - 0s 4ms/step - loss: 106.5617 - val_loss: 42.8884
Epoch 7/100
119/119 [==============================] - 0s 4ms/step - loss: 100.2371 - val_loss: 40.3527
Epoch 8/100
119/119 [==============================] - 0s 4ms/step - loss: 90.4802 - val_loss: 38.2295
Epoch 9/100
119/119 [==============================] - 0s 4ms/step - loss: 86.1673 - val_loss: 36.3941
Epoch 10/100
119/119

120/120 [==============================] - 0s 4ms/step - loss: 30.0437 - val_loss: 13.8173
Epoch 28/100
120/120 [==============================] - 0s 4ms/step - loss: 28.4610 - val_loss: 13.1481
Epoch 29/100
120/120 [==============================] - 0s 3ms/step - loss: 27.3631 - val_loss: 12.5347
Epoch 30/100
120/120 [==============================] - 0s 4ms/step - loss: 25.9368 - val_loss: 12.0038
Epoch 31/100
120/120 [==============================] - 0s 4ms/step - loss: 24.6498 - val_loss: 11.5166
Epoch 32/100
120/120 [==============================] - 0s 4ms/step - loss: 24.2397 - val_loss: 11.1016
Epoch 33/100
120/120 [==============================] - 0s 4ms/step - loss: 23.0941 - val_loss: 10.7186
Epoch 34/100
120/120 [==============================] - 0s 4ms/step - loss: 22.4145 - val_loss: 10.3699
Epoch 35/100
120/120 [==============================] - 0s 4ms/step - loss: 21.5311 - val_loss: 10.0347
Epoch 36/100
120/120 [==============================] - 0s 3ms/step - loss: 2

Train on 120 samples, validate on 14 samples
Epoch 1/100
120/120 [==============================] - 304s 3s/step - loss: 208.3883 - val_loss: 76.8543
Epoch 2/100
120/120 [==============================] - 0s 4ms/step - loss: 180.4363 - val_loss: 67.0955
Epoch 3/100
120/120 [==============================] - 0s 4ms/step - loss: 148.9291 - val_loss: 60.0527
Epoch 4/100
120/120 [==============================] - 0s 4ms/step - loss: 130.0621 - val_loss: 54.5877
Epoch 5/100
120/120 [==============================] - 0s 4ms/step - loss: 118.1137 - val_loss: 50.4621
Epoch 6/100
120/120 [==============================] - 0s 4ms/step - loss: 109.6684 - val_loss: 47.0135
Epoch 7/100
120/120 [==============================] - 0s 4ms/step - loss: 100.1181 - val_loss: 44.3046
Epoch 8/100
120/120 [==============================] - 0s 4ms/step - loss: 92.1743 - val_loss: 41.9887
Epoch 9/100
120/120 [==============================] - 0s 4ms/step - loss: 85.8550 - val_loss: 40.0054
Epoch 10/100
120/120

Epoch 32/100
120/120 [==============================] - 0s 4ms/step - loss: 27.3837 - val_loss: 13.5370
Epoch 33/100
120/120 [==============================] - 0s 3ms/step - loss: 26.4340 - val_loss: 13.0601
Epoch 34/100
120/120 [==============================] - 0s 3ms/step - loss: 25.5894 - val_loss: 12.6186
Epoch 35/100
120/120 [==============================] - 0s 3ms/step - loss: 24.7867 - val_loss: 12.2133
Epoch 36/100
120/120 [==============================] - 0s 4ms/step - loss: 23.9573 - val_loss: 11.8412
Epoch 37/100
120/120 [==============================] - 0s 3ms/step - loss: 22.7870 - val_loss: 11.4911
Epoch 38/100
120/120 [==============================] - 0s 3ms/step - loss: 22.2118 - val_loss: 11.1660
Epoch 39/100
120/120 [==============================] - 0s 3ms/step - loss: 21.9445 - val_loss: 10.8648
Epoch 40/100
120/120 [==============================] - 0s 4ms/step - loss: 21.0757 - val_loss: 10.5883
Epoch 41/100
120/120 [==============================] - 0s 4ms/s

Train on 120 samples, validate on 14 samples
Epoch 1/100
120/120 [==============================] - 307s 3s/step - loss: 171.8049 - val_loss: 65.0204
Epoch 2/100
120/120 [==============================] - 0s 4ms/step - loss: 146.3059 - val_loss: 58.4570
Epoch 3/100
120/120 [==============================] - 0s 4ms/step - loss: 130.8627 - val_loss: 53.3348
Epoch 4/100
120/120 [==============================] - 0s 3ms/step - loss: 115.6729 - val_loss: 49.2498
Epoch 5/100
120/120 [==============================] - 0s 3ms/step - loss: 107.0547 - val_loss: 45.9965
Epoch 6/100
120/120 [==============================] - 0s 4ms/step - loss: 98.5655 - val_loss: 43.2715
Epoch 7/100
120/120 [==============================] - 0s 3ms/step - loss: 91.7803 - val_loss: 41.0375
Epoch 8/100
120/120 [==============================] - 0s 3ms/step - loss: 84.8411 - val_loss: 39.1870
Epoch 9/100
120/120 [==============================] - 0s 4ms/step - loss: 79.7629 - val_loss: 37.6027
Epoch 10/100
120/120 [